In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [2]:
with open('C:\\Users\\NiKordzakhia\\Desktop\\Transformer-BasedGPTModel\\vefxistyaosani.txt', 'r', encoding='utf-8') as f:
  data = f.read()

In [3]:
print(f'Length of dataset in characters: {len(data)}')

Length of dataset in characters: 329322


In [4]:
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !",-.:;?«»აბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰ–—“”
49


In [5]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[c] for c in l])

In [6]:
import torch
import torch.nn as nn
encoded_data = encode(data)
data = torch.tensor(encoded_data, dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([329322]) torch.int64
tensor([17, 16, 32, 42, 20, 29, 30, 35, 12, 25, 29, 12, 24, 20,  0, 15, 12, 29,
        12, 40, 35, 20, 29, 20,  0,  0, 28, 25, 23, 16, 22, 23, 12, 24,  1, 36,
        16, 33, 23, 24, 12,  1, 29, 12, 23, 35, 12, 28, 25,  1, 39, 12, 22, 20,
        19, 12,  1, 23, 20, 19,  1, 39, 22, 20, 16, 28, 20, 19, 12,  4,  0, 18,
        16, 14, 12, 28, 15, 23, 25,  1, 12, 28, 29, 24, 20,  1, 29, 31, 22, 20,
        19, 12,  1, 35, 17, 24, 12,  1, 18, 16])


In [7]:
n = int(len(data) * 0.9)
train_data = data[:n]
valid_data = data[n:]

In [9]:
block_size = 8
train_data[1:block_size + 1], train_data[:block_size]

(tensor([16, 32, 42, 20, 29, 30, 35, 12]),
 tensor([17, 16, 32, 42, 20, 29, 30, 35]))

In [26]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f"Context: {context}; Target: {target}.")

Context: tensor([17]); Target: 16.
Context: tensor([17, 16]); Target: 32.
Context: tensor([17, 16, 32]); Target: 42.
Context: tensor([17, 16, 32, 42]); Target: 20.
Context: tensor([17, 16, 32, 42, 20]); Target: 29.
Context: tensor([17, 16, 32, 42, 20, 29]); Target: 30.
Context: tensor([17, 16, 32, 42, 20, 29, 30]); Target: 35.
Context: tensor([17, 16, 32, 42, 20, 29, 30, 35]); Target: 12.


1. get actual data based on split.
2. get random indeces based on batch size. tensor([264060, 186693])
3. create list of data for x started from index and ended by block size given by previous part.
4. do same for y but moved by one.
5. return x and y data in batches.

In [25]:
torch.manual_seed(1)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else valid_data
  ix = torch.randint(len(data) - block_size, (batch_size, )) # torch.randint(high, size)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')

In [26]:
xb, yb

(tensor([[12,  1, 12, 42, 15, 12,  1, 36],
         [40, 35, 20, 24, 16,  4,  0, 28],
         [20, 24, 12, 42, 16,  4,  1,  5],
         [ 1, 31, 18, 25, 23, 25,  4,  1]]),
 tensor([[ 1, 12, 42, 15, 12,  1, 36, 12],
         [35, 20, 24, 16,  4,  0, 28, 12],
         [24, 12, 42, 16,  4,  1,  5,  1],
         [31, 18, 25, 23, 25,  4,  1, 29]]))

In [29]:
for i in range(batch_size):
  for j in range(block_size):
    context = xb[i, :j+1]
    target = yb[i,j]
    print(f'When the input is {context.tolist()}; target is {target}')

When the input is [12]; target is 1
When the input is [12, 1]; target is 12
When the input is [12, 1, 12]; target is 42
When the input is [12, 1, 12, 42]; target is 15
When the input is [12, 1, 12, 42, 15]; target is 12
When the input is [12, 1, 12, 42, 15, 12]; target is 1
When the input is [12, 1, 12, 42, 15, 12, 1]; target is 36
When the input is [12, 1, 12, 42, 15, 12, 1, 36]; target is 12
When the input is [40]; target is 35
When the input is [40, 35]; target is 20
When the input is [40, 35, 20]; target is 24
When the input is [40, 35, 20, 24]; target is 16
When the input is [40, 35, 20, 24, 16]; target is 4
When the input is [40, 35, 20, 24, 16, 4]; target is 0
When the input is [40, 35, 20, 24, 16, 4, 0]; target is 28
When the input is [40, 35, 20, 24, 16, 4, 0, 28]; target is 12
When the input is [20]; target is 24
When the input is [20, 24]; target is 12
When the input is [20, 24, 12]; target is 42
When the input is [20, 24, 12, 42]; target is 16
When the input is [20, 24, 12,

In [27]:
torch.zeros(5)

tensor([0., 0., 0., 0., 0.])

In [31]:
token_embedding_table = nn.Embedding(vocab_size, 2)
token_embedding_table(xb[:2,:])

tensor([[[ 0.0612, -0.6177],
         [-0.9798, -1.6091],
         [ 0.0612, -0.6177],
         [ 0.6155,  0.1938],
         [ 0.2753,  1.7163],
         [ 0.0612, -0.6177],
         [-0.9798, -1.6091],
         [-0.4610, -0.5601]],

        [[-1.4344, -0.5008],
         [-1.4504, -1.1802],
         [ 2.3571, -1.0373],
         [ 0.7626,  0.4415],
         [ 0.1991,  0.0457],
         [-0.2223,  1.6871],
         [-0.1002, -0.6092],
         [ 1.5392, -0.8696]]], grad_fn=<EmbeddingBackward0>)

### The Bigram Language Model in code is a simple neural network that learns character-level bigram probabilities, meaning it predicts the next character based on the current character.

In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1)

class BiggramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets = None):
    logits = self.token_embedding_table(idx) # (B, T, C) ==> Batch, Time, Channels; C ==> Number of classes (vocabulary size for classification).
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      #print("Logits shape: ", logits.shape)
      #print("Targets shape: ", targets.shape)
      #print('-----------------------------')
      logits = logits.view(B*T, C)
      #print("Logits shape after: ", logits.shape)
      targets = targets.view(B*T)
      #print("Targets shape after: ", targets.shape)
      loss = F.cross_entropy(logits, targets) # Expects (B, C, T) ==> CrossEntropy Loss computes loss per token, then averages over all 32 tokens.
    return logits, loss
  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      print("Logits before: ", logits.shape)
      logits = logits[:, -1,:] # Becomes (B, C) we need last token logits because when sentence comes we do not care about history but what was last word instead.
      #The model generates text one token at a time, and each token generated is influenced by the previous tokens. But for each step, the context for generating the next token is only the most recent token—not the entire sequence.
      print("Logits after: ", logits.shape)
      probs = F.softmax(logits, -1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) ; If probs = [0.1, 0.4, 0.3, 0.2], then calling torch.multinomial(probs, num_samples=1) will randomly select one index from this list (e.g., index 1, index 2, etc.), with the likelihood of each index being selected proportional to its probability in the list.
      idx = torch.cat((idx, idx_next), dim = 1)
    return idx

In [33]:
m = BiggramLanguageModel(vocab_size)
logits, loss = m.forward(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 49])
tensor(4.2583, grad_fn=<NllLossBackward0>)


In [34]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens = 2)[0].tolist()))

Logits before:  torch.Size([1, 1, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 2, 49])
Logits after:  torch.Size([1, 49])

;



In [42]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [43]:
batch_size = 32
for step in range(1000):
  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  print(loss.item())

2.6647534370422363
2.66668438911438
2.701420783996582
2.6608426570892334
2.5892062187194824
2.705380916595459
2.6601386070251465
2.6083102226257324
2.5851805210113525
2.7213497161865234
2.6728696823120117
2.57441782951355
2.540809154510498
2.6013922691345215
2.6145401000976562
2.6092138290405273
2.592005729675293
2.6003780364990234
2.632342576980591
2.6856095790863037
2.4731638431549072
2.5816640853881836
2.6549084186553955
2.5718510150909424
2.5235109329223633
2.6064047813415527
2.5928895473480225
2.6298866271972656
2.633268117904663
2.725774049758911
2.640892267227173
2.6447768211364746
2.6200788021087646
2.6531686782836914
2.6299819946289062
2.663271903991699
2.6244988441467285
2.6399857997894287
2.7188618183135986
2.6941730976104736
2.527024030685425
2.5956368446350098
2.5865156650543213
2.6412501335144043
2.5899643898010254
2.6409332752227783
2.537276268005371
2.6064658164978027
2.65381121635437
2.581110954284668
2.5979928970336914
2.6407806873321533
2.6533782482147217
2.621339082

In [44]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens = 100)[0].tolist()))

Logits before:  torch.Size([1, 1, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 2, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 3, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 4, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 5, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 6, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 7, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 8, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 9, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 10, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 11, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 12, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 13, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 14,

# The mathematical trick in self-attention

In [45]:
torch.manual_seed(1)
B, T, C = 4, 8, 2
# have 4 sequences (B=4).
#Each sequence has 8 tokens (T=8).
#Each token has 2 possible logits (C=2, vocab size) predicting the next character.
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [46]:
xbow = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b, t] = torch.mean(xprev, 0)

### Each row in xbow is average of previous values

In [47]:
x[0]

tensor([[-1.5256, -0.7502],
        [-0.6540, -1.6095],
        [-0.1002, -0.6092],
        [-0.9798, -1.6091],
        [-0.7121,  0.3037],
        [-0.7773, -0.2515],
        [-0.2223,  1.6871],
        [ 0.2284,  0.4676]])

In [48]:
xbow[0]

tensor([[-1.5256, -0.7502],
        [-1.0898, -1.1799],
        [-0.7599, -0.9896],
        [-0.8149, -1.1445],
        [-0.7943, -0.8549],
        [-0.7915, -0.7543],
        [-0.7102, -0.4055],
        [-0.5929, -0.2964]])

In [49]:
torch.manual_seed(1)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()

c = a @ b

print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[5., 9.],
        [4., 8.],
        [3., 3.]])
--
c=
tensor([[12., 20.],
        [12., 20.],
        [12., 20.]])


In [50]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [51]:
torch.manual_seed(1)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()

c = a @ b

print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[5., 9.],
        [4., 8.],
        [3., 3.]])
--
c=
tensor([[ 5.,  9.],
        [ 9., 17.],
        [12., 20.]])


In [56]:
torch.manual_seed(1)
a = torch.tril(torch.ones(3, 3))
print('a previously = ')
print(a)
print('torch.sum(a, 1, keepdim=True) = ')
print(torch.sum(a, 1, keepdim=True))
a = a/torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()

c = a @ b

print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a previously = 
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
torch.sum(a, 1, keepdim=True) = 
tensor([[1.],
        [2.],
        [3.]])
a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[5., 9.],
        [4., 8.],
        [3., 3.]])
--
c=
tensor([[5.0000, 9.0000],
        [4.5000, 8.5000],
        [4.0000, 6.6667]])


In [57]:
# version 1
xbow = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b, t] = torch.mean(xprev, 0)

In [77]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
print('wei=')
print(wei, wei.shape)
print('x=')
print(x, x.shape)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -----> (B, T, C)
print(xbow2, xbow2.shape)

wei=
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]]) torch.Size([8, 8])
x=
tensor([[[-1.5256, -0.7502],
         [-0.6540, -1.6095],
         [-0.1002, -0.6092],
         [-0.9798, -1.6091],
         [-0.7121,  0.3037],
         [-0.7773, -0.2515],
         [-0.2223,  1.6871],
         [ 0.2284,  0.4676]],

        [[-0.6970, -1.1608],
         [ 0.6995,  0.1991],
         [ 0.8657,  0.2444],
         [-0.6629,  0.8073],
         [ 1.1017, -0.1759],
 

In [59]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [63]:
xbow[0]

tensor([[-1.5256, -0.7502],
        [-1.0898, -1.1799],
        [-0.7599, -0.9896],
        [-0.8149, -1.1445],
        [-0.7943, -0.8549],
        [-0.7915, -0.7543],
        [-0.7102, -0.4055],
        [-0.5929, -0.2964]])

In [62]:
xbow2[0]

tensor([[-1.5256, -0.7502],
        [-1.0898, -1.1799],
        [-0.7599, -0.9896],
        [-0.8149, -1.1445],
        [-0.7943, -0.8549],
        [-0.7915, -0.7543],
        [-0.7102, -0.4055],
        [-0.5929, -0.2964]])

In [73]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [74]:
wei = F.softmax(wei, dim = -1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [71]:
# version 3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)
xbow3 = wei @ x

In [67]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [66]:
xbow3

tensor([[[-1.5256e+00, -7.5023e-01],
         [-1.0898e+00, -1.1799e+00],
         [-7.5991e-01, -9.8964e-01],
         [-8.1488e-01, -1.1445e+00],
         [-7.9433e-01, -8.5486e-01],
         [-7.9150e-01, -7.5429e-01],
         [-7.1018e-01, -4.0552e-01],
         [-5.9285e-01, -2.9637e-01]],

        [[-6.9697e-01, -1.1608e+00],
         [ 1.2850e-03, -4.8084e-01],
         [ 2.8942e-01, -2.3909e-01],
         [ 5.1338e-02,  2.2508e-02],
         [ 2.6141e-01, -1.7181e-02],
         [-1.5642e-01, -2.5539e-01],
         [-1.2534e-01, -3.0716e-01],
         [-2.0943e-01, -2.8522e-01]],

        [[ 1.8793e+00, -7.2132e-02],
         [ 1.0186e+00, -4.2279e-01],
         [ 7.4539e-01, -2.6663e-01],
         [ 5.9728e-01, -3.1889e-01],
         [ 4.5562e-01, -1.9657e-01],
         [ 3.5338e-01, -1.6860e-01],
         [ 6.3963e-01, -2.9271e-01],
         [ 7.5652e-01, -3.3485e-01]],

        [[-9.2739e-01,  5.4514e-01],
         [-4.3056e-01,  5.4051e-02],
         [-3.2837e-02,  1.8320e-

In [84]:
# version 4: self-attention!
torch.manual_seed(1)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1) #(B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril ==0, float('-inf')) # If we do decoder block we have this line but if we would encoder block then we do not need this line.
wei = F.softmax(wei, -1)

v = value(x)

out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [85]:
wei[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.5482e-01, 4.5181e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.5075e-01, 2.4786e-01, 6.0139e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [6.0910e-01, 6.0784e-02, 1.3287e-01, 1.9725e-01, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.1235e-01, 9.3733e-04, 1.5950e-01, 2.2520e-01, 5.0200e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.1421e-02, 2.6164e-02, 3.0799e-01, 2.3788e-01, 3.0135e-01, 4.5193e-02,
         0.0000e+00, 0.0000e+00],
        [5.5458e-02, 3.9424e-01, 3.3161e-02, 3.7227e-02, 2.6962e-02, 2.5549e-01,
         1.9746e-01, 0.0000e+00],
        [4.4814e-01, 1.0565e-02, 1.0351e-01, 4.2752e-02, 6.5090e-02, 4.0655e-02,
         2.8155e-01, 7.7372e-03]], grad_fn=<SelectBackward0>)

In [91]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)# * head_size ** -0.5

In [92]:
k.var()

tensor(1.0965)

In [93]:
q.var()

tensor(1.1010)

In [94]:
wei.var()

tensor(16.6169)